<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dara Loading and Preprocessing</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Conclustions</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# 01 - ***ML for Texts*** - Project for WikiShop

[Nice reading about BERT](https://docs.deeppavlov.ai/en/master/features/models/bert.html)

BERT (Bidirectional Encoder Representations from Transformers) is a Transformer pre-trained on masked language model and next sentence prediction tasks. This approach showed state-of-the-art results on a wide range of NLP tasks in English.

### Project Goal
The project goal is a binary text classification using BERT model.

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [107]:
!pip3 install -q transformers


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [2]:
import sys
!{sys.executable} -m pip install spacy -q
!{sys.executable} -m spacy download en_core_web_sm -q

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/pip/__main__.py", line 9, in <module>
    if sys.path[0] in ("", os.getcwd()):
PermissionError: [Errno 1] Operation not permitted
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
2022-11-30 20:54:17.459138: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to us

In [98]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
import re
import spacy
from tqdm import notebook
from sklearn.model_selection import train_test_split

# regression problems
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier
from sklearn.dummy import DummyRegressor
import lightgbm as lgb

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,make_scorer
from catboost import CatBoostClassifier
import torch
import transformers
from tqdm import notebook

from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

import pandas as pd
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None 

## 1 Подготовка

In [136]:
data = pd.read_csv('/Users/yuliabezginova/PycharmProjects/00_files-for_NLP/toxic_comments.csv', index_col=[0])

In [137]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [138]:
data.toxic.unique()

array([0, 1])

In [139]:
# проверка на пропуски в таргете
data['toxic'].isna().sum()

0

### 1.1 Работаем со стоп-словами

In [140]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuliabezginova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1.3 Переведём тексты в стандартный для Python формат: кодировку Unicode U.

In [144]:
# Изменим кодировку методом astype():
corpus = data['text'].values.astype('U')
# corpus_train = X_train.values.astype('U')
# corpus_test = X_test.values.astype('U')

### 1.4 Создадим функцию для лемматизации и очистки текста

In [145]:
def lemmatize(text):
    m = Mystem()
    m = re.sub(r'[^a-zA-Z ]', ' ', text)
    # объединим элементы в одну строку пробелом или без него
    lem = ' '.join(m.split())
    return lem

#### Протестируем работу функции лемматизации и очистки текста.

In [147]:
print("Исходный текст:", corpus[181])
print("Очищенный и лемматизированный текст:", lemmatize(corpus[181]))

Исходный текст: you are a stupid fuck 

and your mother's cunt stinks
Очищенный и лемматизированный текст: you are a stupid fuck and your mother s cunt stinks


#### Применим функцию для лемматизации и очистки текста

In [148]:
data['lem_text'] = data.text.apply(lemmatize)

In [149]:
data.head()

,text,toxic,lem_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


### 1.1 Делим данные на тестовую и обучающую выборки

In [150]:
features = data.drop(columns='toxic', axis=1)
target = data['toxic']

In [151]:
features.shape

(159292, 2)

In [152]:
target.shape

(159292,)

In [112]:
# Разделяю на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                    test_size=0.2, 
                                                    random_state=5)

In [113]:
print('Train features sample size:', X_train.shape[0])
print('Train target sample size:', y_train.shape[0])
print()
print('Test features sample size:', X_test.shape[0])
print('Test target sample size:', y_test.shape[0])
print()
print('Test sample, %:', round(X_test.shape[0] / (X_test.shape[0] + X_train.shape[0]) * 100, 2))

Train features sample size: 127433
Train target sample size: 127433

Test features sample size: 31859
Test target sample size: 31859

Test sample, %: 20.0


### ВЫВОД: Исходные данные разделены на тестовые и обучающие, выделены признаки и таргетные значения. Дальнейшие преобразования текста будем производить только с признаками (текстом).

Применим модель "мешка слов" к набору данных.

### 1.5 Модель «мешка слов» реализована в классе CountVectorizer

Мы импортируем класс CountVectorizer, создам экземпляр класса и подгоняем модель к нашим данным для анализа тональности твитов.

In [153]:
vect = CountVectorizer(stop_words=stopwords)

In [154]:
# Считаю Tf-idf для обучающей
X_train_vect = vect.fit_transform(X_train.lem_text)

# Считаю Tf-idf для тестовой
X_test_vect = vect.transform(X_test.lem_text)

Перед тем как мы пытаемся улучшить выделение признаков, давайте измерим качество модели, построив классификатор. 

У нас есть обучающие метки, хранящиеся в y_train и обучающие данные, представленные в виде «мешка слов» X_train, таким образом, мы можем обучить классификатор по этим данным. 

Как правило, для подобных высокоразмерных разреженных данных лучше всего работают линейные модели типа LogisticRegression.

### 1.6 Масштабирование признаков с помощью TF-IDF

Следующий подход вместо исключения несущественных признаков пытается масштабировать признаки в зависимости от степени их информативности. Одним из наиболее распространенных способов такого масштабирования является метод частота термина-обратная частота документа (term frequency-inverse document frequency, tf-idf). 

**Идея этого метода заключается в том, чтобы присвоить большой вес термину, который часто встречается в конкретном документе, но при этом редко встречается в остальных документах корпуса. Если слово часто появляется в конкретном документе, но при этом редко встречается в остальных документах, оно, вероятно, будет описывать содержимое этого документа лучше.**

Мешок слов учитывает частоту употребления слов. Посмотрим, как часто уникальное слово встречается во всём корпусе и в отдельном его тексте.

Оценка важности слова определяется величиной TF-IDF (от англ. term frequency, «частота терма, или слова»; inverse document frequency, «обратная частота документа, или текста»). То есть TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе.

#### TFIDF = TF * IDF

IDF нужна в формуле, чтобы уменьшить вес слов, наиболее
распространённых в любом другом тексте заданного корпуса.
IDF зависит от общего числа текстов в корпусе (D) и количества
текстов, в которых это слово встречается (d).

Большая величина TF-IDF говорит об уникальности слова в тексте
по отношению к корпусу. Чем чаще оно встречается в конкретном
тексте и реже в остальных, тем выше значение TF-IDF.

**Если данные разделены на обучающую и тестовую выборки,
функцию fit() запустим только на обучающей. Иначе тестирование
будет нечестным: в модели будут учтены частоты слов из тестовой
выборки.**

Сначала переведем выборки X_train, X_test в векторный вид.


In [116]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [155]:
# Считаю Tf-idf для обучающей
tf_idf_X_train = count_tf_idf.fit_transform(X_train['lem_text'])

# Считаю Tf-idf для тестовой
tf_idf_X_test = count_tf_idf.transform(X_test['lem_text'])

In [156]:
print("Размер матрицы TF-IDF X_train:", tf_idf_X_train.shape)
print("Размер матрицы TF-IDF X_test:", tf_idf_X_test.shape)

Размер матрицы TF-IDF X_train: (127433, 147485)
Размер матрицы TF-IDF X_test: (31859, 147485)


### ВЫВОД: Данные очищены от странных букв и знаков, знаки пунктуации и стоп-слова удалены, данные разделены на тестовую и обучающую выборки, в каждой из которой выделены признаки и целевое значение. Выборки признаков приведены к векторному виду и готовы к обучению.

## 2 Обучение

### 2.1 Logistic Regression с использованием перекрестной проверки для модели «мешка слов» в классе CountVectorizer().

In [157]:
scores = cross_val_score(LogisticRegression(), X_train_vect, y_train, cv=10)
print("Средняя правильность перекрестной проверки: {:.2f}".format(np.mean(scores)))

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

Средняя правильность перекрестной проверки: 0.96


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### ВЫВОД: Получаем, что средняя правильность перекрестной проверки: 96%.

Известно, что логистическая регрессия имеет параметр регуляризации C, который мы можем настроить с помощью перекрестной проверки.

In [158]:
logreg = LogisticRegression(random_state=5)

In [159]:
param_grid = [
    {'C': [0.01, 0.1, 1, 2, 10, 100], 
     'penalty': ['l1', 'l2']}
]

In [160]:
logreg_grid = GridSearchCV(logreg, 
                           param_grid, 
                           cv=10, 
                           verbose=False, 
                           n_jobs=-1)

logreg_grid.fit(X_train_vect, y_train)

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

GridSearchCV(cv=10, estimator=LogisticRegression(random_state=5), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 2, 10, 100],
                          'penalty': ['l1', 'l2']}],
             verbose=False)

In [161]:
print("Наилучшее значение перекрестной проверки: {:.2f}".format(logreg_grid.best_score_))
print("Наилучшие параметры: ", logreg_grid.best_params_)

Наилучшее значение перекрестной проверки: 0.96
Наилучшие параметры:  {'C': 2, 'penalty': 'l2'}


Теперь мы можем оценить на тестовом наборе обобщающую способность при использовании данной настройки параметра:

In [162]:
print("Правильность на тестовом наборе: {:.2f}".format(logreg_grid.score(X_test_vect, y_test)))

Правильность на тестовом наборе: 0.96


In [ ]:
y_pred_vect = grid.predict(tf_idf_test)

In [ ]:
confusion_matrix(y_pred_vect, y_test)

In [ ]:
print("F1-мера на тестовой выборке: {:.2f}".format(round(f1_score(y_pred_vect, y_test),2)))

### 2.2 Logistic Regression с использованием перекрестной проверки для модели «мешка слов» в классе TfidfVectorizer().

In [163]:
from sklearn.pipeline import make_pipeline

Поскольку tf-idf фактически использует статистические свойства
обучающих данных, мы воспользуемся конвейером, чтобы убедиться в достоверности результатов решетчатого поиска.

In [164]:
logreg = LogisticRegression(random_state=5)

In [166]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(logreg, param_grid, cv=5)

grid.fit(tf_idf_X_train, y_train)

print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))

/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

Наилучшее значение перекрестной проверки: 0.96


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print("Наилучшее значение перекр проверки: {:.2f}".format(grid.best_score_))

In [ ]:
print("Правильность на тестовом наборе: {:.2f}".format(grid.score(tf_idf_X_test, y_test)))

In [ ]:
y_pred_tfidf = grid.predict(tf_idf_test)

In [ ]:
confusion_matrix(y_pred_tfidf, y_test)

In [ ]:
print("F1-мера на тестовой выборке: {:.2f}".format(round(f1_score(y_pred_tfidf, y_test),2)))

### 2.3 CatBoost в модели "мешка слова" в классе TfidfVectorizer()

In [ ]:
# initiatint the model
catboost = CatBoostClassifier(random_seed=5, 
                              loss_function='Logloss')

# preparing parameters to ask GridSearch, which contains cross validation
parameters_cat = {'iterations':[50], 
                  'learning_rate': np.arange(0.1,1,0.2)}

gsearch_catboost = GridSearchCV(catboost, 
                                param_grid=parameters_cat,
                                verbose=False)

gsearch_catboost.fit(tf_idf_X_train, y_train)

0:	learn: 0.5939473	total: 3.05s	remaining: 2m 29s
1:	learn: 0.5154453	total: 4.68s	remaining: 1m 52s
2:	learn: 0.4532370	total: 5.87s	remaining: 1m 31s
3:	learn: 0.4068591	total: 6.99s	remaining: 1m 20s
4:	learn: 0.3720388	total: 8.09s	remaining: 1m 12s
5:	learn: 0.3437147	total: 9.2s	remaining: 1m 7s
6:	learn: 0.3206467	total: 10.3s	remaining: 1m 3s
7:	learn: 0.3029510	total: 11.4s	remaining: 59.9s
8:	learn: 0.2886673	total: 12.6s	remaining: 57.3s
9:	learn: 0.2760725	total: 13.7s	remaining: 54.9s
10:	learn: 0.2669043	total: 14.8s	remaining: 52.7s
11:	learn: 0.2594292	total: 16s	remaining: 50.5s
12:	learn: 0.2533731	total: 17s	remaining: 48.5s
13:	learn: 0.2484205	total: 18.1s	remaining: 46.6s
14:	learn: 0.2440143	total: 19.2s	remaining: 44.8s
15:	learn: 0.2400490	total: 20.4s	remaining: 43.3s
16:	learn: 0.2370516	total: 21.5s	remaining: 41.7s
17:	learn: 0.2336507	total: 22.6s	remaining: 40.2s
18:	learn: 0.2309671	total: 23.8s	remaining: 38.8s
19:	learn: 0.2285956	total: 24.8s	remaini

13:	learn: 0.2479260	total: 17.1s	remaining: 44s
14:	learn: 0.2434892	total: 18.2s	remaining: 42.5s
15:	learn: 0.2389627	total: 19.3s	remaining: 41s
16:	learn: 0.2354247	total: 20.4s	remaining: 39.6s
17:	learn: 0.2322560	total: 21.6s	remaining: 38.3s
18:	learn: 0.2298084	total: 22.7s	remaining: 37.1s
19:	learn: 0.2273066	total: 24s	remaining: 36s
20:	learn: 0.2247383	total: 25.3s	remaining: 34.9s
21:	learn: 0.2224923	total: 26.5s	remaining: 33.7s
22:	learn: 0.2206985	total: 28.2s	remaining: 33.1s
23:	learn: 0.2193029	total: 29.8s	remaining: 32.3s
24:	learn: 0.2178802	total: 31.2s	remaining: 31.2s
25:	learn: 0.2164012	total: 32.6s	remaining: 30.1s
26:	learn: 0.2150215	total: 34s	remaining: 28.9s
27:	learn: 0.2134995	total: 35.2s	remaining: 27.7s
28:	learn: 0.2123366	total: 36.9s	remaining: 26.7s
29:	learn: 0.2110873	total: 38.8s	remaining: 25.9s
30:	learn: 0.2098515	total: 40.6s	remaining: 24.9s
31:	learn: 0.2085974	total: 42.3s	remaining: 23.8s
32:	learn: 0.2072060	total: 44.6s	remaini

26:	learn: 0.1742533	total: 36.9s	remaining: 31.4s
27:	learn: 0.1733712	total: 38.7s	remaining: 30.4s
28:	learn: 0.1721921	total: 40.2s	remaining: 29.1s
29:	learn: 0.1711671	total: 41.6s	remaining: 27.7s
30:	learn: 0.1702404	total: 42.9s	remaining: 26.3s
31:	learn: 0.1689780	total: 44.1s	remaining: 24.8s
32:	learn: 0.1680857	total: 45.3s	remaining: 23.3s
33:	learn: 0.1671419	total: 46.7s	remaining: 22s
34:	learn: 0.1662193	total: 47.9s	remaining: 20.5s
35:	learn: 0.1655198	total: 49s	remaining: 19.1s
36:	learn: 0.1647397	total: 50.2s	remaining: 17.6s
37:	learn: 0.1637731	total: 51.3s	remaining: 16.2s
38:	learn: 0.1631061	total: 52.5s	remaining: 14.8s
39:	learn: 0.1621352	total: 53.6s	remaining: 13.4s
40:	learn: 0.1615614	total: 54.8s	remaining: 12s
41:	learn: 0.1602077	total: 56.2s	remaining: 10.7s
42:	learn: 0.1595407	total: 57.3s	remaining: 9.32s
43:	learn: 0.1586420	total: 58.3s	remaining: 7.95s
44:	learn: 0.1578166	total: 59.5s	remaining: 6.61s
45:	learn: 0.1571014	total: 1m	remain

39:	learn: 0.1631462	total: 45.1s	remaining: 11.3s
40:	learn: 0.1624061	total: 46.2s	remaining: 10.1s
41:	learn: 0.1616650	total: 47.2s	remaining: 9s
42:	learn: 0.1607180	total: 48.3s	remaining: 7.87s
43:	learn: 0.1601071	total: 49.4s	remaining: 6.74s
44:	learn: 0.1589671	total: 50.5s	remaining: 5.61s
45:	learn: 0.1581788	total: 51.6s	remaining: 4.48s
46:	learn: 0.1573204	total: 52.7s	remaining: 3.36s
47:	learn: 0.1566901	total: 53.8s	remaining: 2.24s
48:	learn: 0.1559048	total: 54.8s	remaining: 1.12s
49:	learn: 0.1551892	total: 55.9s	remaining: 0us
0:	learn: 0.3464329	total: 1.55s	remaining: 1m 16s
1:	learn: 0.2614594	total: 2.81s	remaining: 1m 7s
2:	learn: 0.2376437	total: 3.96s	remaining: 1m 2s
3:	learn: 0.2220583	total: 5.07s	remaining: 58.3s
4:	learn: 0.2135715	total: 6.16s	remaining: 55.5s
5:	learn: 0.2074178	total: 7.25s	remaining: 53.2s
6:	learn: 0.2025665	total: 8.34s	remaining: 51.2s
7:	learn: 0.1984095	total: 9.43s	remaining: 49.5s
8:	learn: 0.1945071	total: 10.5s	remaining:

2:	learn: 0.2362300	total: 3.77s	remaining: 59s
3:	learn: 0.2222085	total: 4.93s	remaining: 56.7s
4:	learn: 0.2129957	total: 6.04s	remaining: 54.4s
5:	learn: 0.2072191	total: 7.13s	remaining: 52.3s
6:	learn: 0.2019202	total: 8.49s	remaining: 52.2s
7:	learn: 0.1967159	total: 9.64s	remaining: 50.6s
8:	learn: 0.1930191	total: 10.9s	remaining: 49.7s
9:	learn: 0.1896643	total: 12.5s	remaining: 50s
10:	learn: 0.1868666	total: 13.9s	remaining: 49.2s
11:	learn: 0.1834087	total: 15.2s	remaining: 48.2s
12:	learn: 0.1809649	total: 16.5s	remaining: 46.9s
13:	learn: 0.1776502	total: 17.9s	remaining: 46s
14:	learn: 0.1746658	total: 19.3s	remaining: 45s
15:	learn: 0.1729971	total: 20.6s	remaining: 43.8s
16:	learn: 0.1708787	total: 22s	remaining: 42.7s
17:	learn: 0.1692775	total: 23.4s	remaining: 41.6s
18:	learn: 0.1674295	total: 24.8s	remaining: 40.4s
19:	learn: 0.1656833	total: 26.1s	remaining: 39.2s
20:	learn: 0.1642344	total: 27.5s	remaining: 37.9s
21:	learn: 0.1624728	total: 28.9s	remaining: 36.8

In [ ]:
# # CatBoost
best_score_catboost = gsearch_catboost.best_score_
print('Best score CatBoostRegressor: {}'.format(best_score_catboost))


## 3 Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны